### **IMPORTAR BIBLIOTECAS**

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from functools import reduce
from pyspark.sql import DataFrame

StatementMeta(, 31ffabd4-67ed-4e1d-94de-ce8fa2b0e87f, 3, Finished, Available, Finished)

### **PROCESSAR DADOS**

In [2]:
files = mssparkutils.fs.ls('Files/bronze/dados')

files = [file.name for file in files]

files = files[-3:]

print(files)

StatementMeta(, 31ffabd4-67ed-4e1d-94de-ce8fa2b0e87f, 4, Finished, Available, Finished)

['PNAD_COVID_092020.csv', 'PNAD_COVID_102020.csv', 'PNAD_COVID_112020.csv']


In [3]:
all_data = []

for file in files:
    path = f'Files/bronze/dados/{file}'
    
    df = spark.read \
          .format('csv') \
          .option('header', 'true') \
          .load(path) \
          .select('Ano', 'UF', 'V1013', 'A002', 'A003', 'A004', 'A005', 'B009B', 'B009D', 'B009F', 'C007C', 'C01011', 'D0031', 'D0051', 'F001', 'F0022') \
          .withColumnsRenamed(
                {
                    'ANO': 'ano',
                    'UF': 'id',
                    'V1013': 'mes',
                    'A002': 'idade',
                    'A003': 'sexoId',
                    'A004': 'corId',
                    'A005': 'escolaridadeId',
                    'B009B': 'exame_swabId',
                    'B009D': 'exame_dedoId',
                    'B009F': 'exame_sangueId',
                    'C007C': 'trabalhoId',
                    'C01011': 'faixa_rendimentoId',
                    'D0031': 'bolsa_familiaId',
                    'D0051': 'auxilioId',
                    'F001': 'domicilioId',
                    'F0022': 'valor_aluguelId'
                }
            )
        
    df = df.select([col(c).cast('int').alias(c) for c in df.columns])

    all_data.append(df)

df = reduce(DataFrame.unionByName, all_data)

df.show()

StatementMeta(, 31ffabd4-67ed-4e1d-94de-ce8fa2b0e87f, 5, Finished, Available, Finished)

+----+---+---+-----+------+-----+--------------+------------+------------+--------------+----------+------------------+---------------+---------+-----------+---------------+
| ano| id|mes|idade|sexoId|corId|escolaridadeId|exame_swabId|exame_dedoId|exame_sangueId|trabalhoId|faixa_rendimentoId|bolsa_familiaId|auxilioId|domicilioId|valor_aluguelId|
+----+---+---+-----+------+-----+--------------+------------+------------+--------------+----------+------------------+---------------+---------+-----------+---------------+
|2020| 11|  9|   36|     1|    4|             5|        NULL|        NULL|          NULL|        35|                 4|              2|        1|          1|           NULL|
|2020| 11|  9|   30|     2|    4|             7|        NULL|        NULL|          NULL|        27|                 4|              2|        1|          1|           NULL|
|2020| 11|  9|   13|     1|    4|             2|        NULL|        NULL|          NULL|      NULL|              NULL|           

In [4]:
df.write \
  .format('parquet') \
  .mode('overwrite') \
  .save('Files/silver/dados')

print('Dados armazenados na camada silver! 🥈')

StatementMeta(, 31ffabd4-67ed-4e1d-94de-ce8fa2b0e87f, 6, Finished, Available, Finished)

Dados armazenados na camada silver! 🥈
